In [43]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
import cv2
import os
from PIL import Image

PATH_TO_ORIGINAL = 'C:/Users/Владимир/Desktop/Original/'
PATH_TO_AUGMENTED = 'C:/Users/Владимир/Desktop/Augmented/'
PATH_TO_TEST = 'C:/Users/Владимир/Desktop/Testing/'

In [44]:
data_gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                              shear_range=0.1, zoom_range=0.1, fill_mode='constant', horizontal_flip=True)

for file_name in os.listdir(PATH_TO_AUGMENTED):
    file_path = PATH_TO_AUGMENTED + file_name
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as exception:
        print(exception)

for file_name in os.listdir(PATH_TO_ORIGINAL):
    if file_name.endswith('.jpg'):
        img_file = load_img(PATH_TO_ORIGINAL + file_name, target_size=(210, 260))
        img_array = np.expand_dims(img_to_array(img_file), axis=0)

        aug_count = 1
        for batch in data_gen.flow(img_array, batch_size=1, save_to_dir=PATH_TO_AUGMENTED,
                                   save_prefix=os.path.splitext(file_name)[0] + '_aug', save_format='jpg'):
            aug_count += 1
            if aug_count > 100:
                break

In [45]:
face_cascades = cv2.CascadeClassifier('cascade_frontal_face_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create(1, 8, 8, 8, 123)

images, labels = [], []
image_paths = [os.path.join(PATH_TO_AUGMENTED, file_name) for file_name in os.listdir(PATH_TO_AUGMENTED)]

if face_cascades.empty():
    raise Exception('cascade is empty')

labels_to_names = {0: 'chaikovsky', 1: 'rahmaninov', 2: 'shostakovich'}

for image_path in image_paths:
    image = np.array(Image.open(image_path).convert('L'), 'uint8')
    person_label = int(os.path.split(image_path)[1].split('_')[0])

    face = face_cascades.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x_margin, y_margin, face_width, face_height) in face:
        images.append(image[y_margin: y_margin + face_height, x_margin: x_margin + face_width])
        labels.append(person_label)

        cv2.imshow('', image[y_margin: y_margin + face_height, x_margin: x_margin + face_width])
        cv2.waitKey(50)

cv2.destroyAllWindows()
recognizer.train(images, np.array(labels))

In [46]:
image_paths = [os.path.join(PATH_TO_TEST, file_name) for file_name in os.listdir(PATH_TO_TEST)]

for image_path in image_paths:
    image = np.array(Image.open(image_path).convert('L'), 'uint8')
    face = face_cascades.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x_margin, y_margin, face_width, face_height) in face:
        person_label_predicted, conf = \
            recognizer.predict(image[y_margin: y_margin + face_height, x_margin: x_margin + face_width])

        person_label_actual = int(os.path.split(image_path)[1].split('_')[0])
        if person_label_actual == person_label_predicted:
            print('{} is correctly recognized with confidence {}'
                  .format(labels_to_names[person_label_actual], conf))
        else:
            print('{} is incorrectly recognized as {}'.format(labels_to_names[person_label_actual],
                                                              labels_to_names[person_label_predicted]))
        cv2.imshow('recognizing face', image[y_margin: y_margin + face_height, x_margin: x_margin + face_width])
        cv2.waitKey(2000)
        
cv2.destroyAllWindows()

chaikovsky is correctly recognized with confidence 52.64853440291559
chaikovsky is correctly recognized with confidence 57.269449550377985
rahmaninov is correctly recognized with confidence 57.819462550064145
rahmaninov is correctly recognized with confidence 70.42716694709705
shostakovich is correctly recognized with confidence 65.05135226667434
shostakovich is correctly recognized with confidence 60.112786579542


In [47]:
recognizer.write('Models/recognizer.xml')

In [48]:
updated_recognizer = cv2.face.LBPHFaceRecognizer_create(1, 8, 8, 8, 123)

In [58]:
updated_recognizer.read('Models/recognizer.xml')

In [59]:
image_path = 'C:/Users/Владимир/Desktop/Testing/0_chaikovsky_A.jpg'

image = np.array(Image.open(image_path).convert('L'), 'uint8')
face = face_cascades.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

for (x_margin, y_margin, face_width, face_height) in face:
    person_label_predicted, conf = \
        updated_recognizer.predict(image[y_margin: y_margin + face_height, x_margin: x_margin + face_width])

    person_label_actual = int(os.path.split(image_path)[1].split('_')[0])
    if person_label_actual == person_label_predicted:
        print('{} is correctly recognized with confidence {}'
              .format(labels_to_names[person_label_actual], conf))
    else:
        print('{} is incorrectly recognized as {}'.format(labels_to_names[person_label_actual],
                                                          labels_to_names[person_label_predicted]))
    cv2.imshow('recognizing face', image[y_margin: y_margin + face_height, x_margin: x_margin + face_width])
    cv2.waitKey(2000)
    
cv2.destroyAllWindows()

chaikovsky is correctly recognized with confidence 52.64853440291559
